In [1]:
import polars as pl
import numpy as np
import random
from datetime import datetime, timedelta

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [2]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [3]:
num_cars = 1000
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
print(data.head())

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SLICE[offset: 0, len: 5]
  DF ["car_id", "timestamp", "latitude", "longitude"]; PROJECT */5 COLUMNS; SELECTION: "None"


In [4]:
print(data.collect())

shape: (6_296_894, 5)
┌─────────┬────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id  ┆ timestamp                  ┆ latitude  ┆ longitude   ┆ notification  │
│ ---     ┆ ---                        ┆ ---       ┆ ---         ┆ ---           │
│ str     ┆ str                        ┆ f64       ┆ f64         ┆ str           │
╞═════════╪════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0   ┆ 2023-01-01T00:00:00        ┆ 27.37124  ┆ -100.629631 ┆ low_fuel      │
│ car_0   ┆ 2023-01-01T00:30:00        ┆ 27.374879 ┆ -100.638245 ┆ engine_check  │
│ car_0   ┆ 2023-01-01T01:00:00        ┆ 27.373578 ┆ -100.632955 ┆ low_fuel      │
│ car_0   ┆ 2023-01-01T01:19:14.797138 ┆ 27.373578 ┆ -100.632955 ┆ null          │
│ car_0   ┆ 2023-01-01T01:21:13.087529 ┆ 27.373578 ┆ -100.632955 ┆ tire_pressure │
│ …       ┆ …                          ┆ …         ┆ …           ┆ …             │
│ car_999 ┆ 2023-01-08T00:01:50.380764 ┆ 25.872271 ┆ -98.759214  

## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [5]:
data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime))

print(data.collect())

shape: (6_296_894, 5)
┌─────────┬────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id  ┆ timestamp                  ┆ latitude  ┆ longitude   ┆ notification  │
│ ---     ┆ ---                        ┆ ---       ┆ ---         ┆ ---           │
│ str     ┆ datetime[μs]               ┆ f64       ┆ f64         ┆ str           │
╞═════════╪════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0   ┆ 2023-01-01 00:00:00        ┆ 27.37124  ┆ -100.629631 ┆ low_fuel      │
│ car_0   ┆ 2023-01-01 00:30:00        ┆ 27.374879 ┆ -100.638245 ┆ engine_check  │
│ car_0   ┆ 2023-01-01 01:00:00        ┆ 27.373578 ┆ -100.632955 ┆ low_fuel      │
│ car_0   ┆ 2023-01-01 01:19:14.797138 ┆ 27.373578 ┆ -100.632955 ┆ null          │
│ car_0   ┆ 2023-01-01 01:21:13.087529 ┆ 27.373578 ┆ -100.632955 ┆ tire_pressure │
│ …       ┆ …                          ┆ …         ┆ …           ┆ …             │
│ car_999 ┆ 2023-01-08 00:01:50.380764 ┆ 25.872271 ┆ -98.759214  

### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es encesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [6]:
data = data.sort(["car_id", "timestamp"], descending=[False, False])
data = data.with_columns([
    pl.col("timestamp").diff().over('car_id').alias("notification_time")
    ])
data.collect()

car_id,timestamp,latitude,longitude,notification,notification_time
str,datetime[μs],f64,f64,str,duration[μs]
"""car_0""",2023-01-01 00:00:00,27.37124,-100.629631,"""low_fuel""",null
"""car_0""",2023-01-01 00:30:00,27.374879,-100.638245,"""engine_check""",30m
"""car_0""",2023-01-01 01:00:00,27.373578,-100.632955,"""low_fuel""",30m
"""car_0""",2023-01-01 01:00:52.394505,27.373578,-100.632955,"""engine_check""",52s 394505µs
"""car_0""",2023-01-01 01:01:42.964122,27.373578,-100.632955,"""tire_pressure""",50s 569617µs
…,…,…,…,…,…
"""car_999""",2023-01-07 23:43:19.480068,25.872271,-98.759214,"""engine_check""",43s 658162µs
"""car_999""",2023-01-07 23:49:31.067630,25.872271,-98.759214,"""low_fuel""",6m 11s 587562µs
"""car_999""",2023-01-07 23:49:37.393064,25.872271,-98.759214,"""low_fuel""",6s 325434µs


2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [7]:

#! pip install geopy
from geopy import distance
def calculate_distance(lat1, lon1, lat2, lon2):
    if None in [lat1, lon1, lat2, lon2]:
        return None
    return distance.distance((lat1, lon1), (lat2, lon2)).meters

data = data.collect()

In [8]:

data = data.with_columns([
    pl.col("latitude").shift(1).over("car_id").alias("prev_latitude"),
    pl.col("longitude").shift(1).over("car_id").alias("prev_longitude"), 
])

df2 = data.map_rows(lambda t: calculate_distance(t[2], t[3], t[6], t[7]))

data = data.with_columns(pl.Series(name="distance", values=df2)) 



3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`


In [9]:
x="5m"
data = data.with_columns(
    pl.col("timestamp").dt.truncate(x).alias("intervalo_manual")
)
data2 = data.groupby(["car_id", "intervalo_manual"])

data2

/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_52801/1993981362.py:5: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  data2 = data.groupby(["car_id", "intervalo_manual"])


In [10]:
stats = data2.agg([
    pl.col("notification_time").dt.seconds().mean().alias("mean"),
    pl.col("notification_time").dt.seconds().median().alias("median"),
    pl.col("notification_time").dt.seconds().var().alias("variance"),
    pl.col("notification_time").dt.seconds().max().alias("max"),
    pl.col("notification_time").dt.seconds().min().alias("min"),
])
stats=stats.sort(["car_id", "intervalo_manual"])
stats

/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_52801/2582669398.py:2: DeprecationWarning: `seconds` is deprecated. It has been renamed to `total_seconds`.
  pl.col("notification_time").dt.seconds().mean().alias("mean"),
/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_52801/2582669398.py:3: DeprecationWarning: `seconds` is deprecated. It has been renamed to `total_seconds`.
  pl.col("notification_time").dt.seconds().median().alias("median"),
/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_52801/2582669398.py:4: DeprecationWarning: `seconds` is deprecated. It has been renamed to `total_seconds`.
  pl.col("notification_time").dt.seconds().var().alias("variance"),
/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_52801/2582669398.py:5: DeprecationWarning: `seconds` is deprecated. It has been renamed to `total_seconds`.
  pl.col("notification_time").dt.seconds().max().alias("max"),
/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_52801/25

car_id,intervalo_manual,mean,median,variance,max,min
str,datetime[μs],f64,f64,f64,i64,i64
"""car_0""",2023-01-01 00:00:00,null,null,null,null,null
"""car_0""",2023-01-01 00:30:00,1800.0,1800.0,null,1800,1800
"""car_0""",2023-01-01 01:00:00,259.5,51.0,388066.0,1800,8
"""car_0""",2023-01-01 01:05:00,81.333333,97.0,2734.333333,124,23
"""car_0""",2023-01-01 01:10:00,107.0,107.0,11858.0,184,30
…,…,…,…,…,…,…
"""car_999""",2023-01-07 23:35:00,53.8,31.0,1287.7,101,26
"""car_999""",2023-01-07 23:40:00,84.333333,97.0,1345.333333,113,43
"""car_999""",2023-01-07 23:45:00,188.5,188.5,66612.5,371,6


In [11]:
stats2 = data2.agg([
    pl.col("distance").mean().alias("mean_distance"),
    pl.col("distance").median().alias("median_distance"),
    pl.col("distance").var().alias("variance_distance"),
    pl.col("distance").max().alias("max_distance"),
    pl.col("distance").min().alias("min_distance"),
])

stats2 = stats2.sort(["car_id", "intervalo_manual"])
stats2


car_id,intervalo_manual,mean_distance,median_distance,variance_distance,max_distance,min_distance
str,datetime[μs],f64,f64,f64,f64,f64
"""car_0""",2023-01-01 00:00:00,null,null,null,null,null
"""car_0""",2023-01-01 00:30:00,942.742388,942.742388,null,942.742388,942.742388
"""car_0""",2023-01-01 01:00:00,67.84482,0.0,36823.357153,542.758563,0.0
"""car_0""",2023-01-01 01:05:00,0.0,0.0,0.0,0.0,0.0
"""car_0""",2023-01-01 01:10:00,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…
"""car_999""",2023-01-07 23:35:00,0.0,0.0,0.0,0.0,0.0
"""car_999""",2023-01-07 23:40:00,0.0,0.0,0.0,0.0,0.0
"""car_999""",2023-01-07 23:45:00,0.0,0.0,0.0,0.0,0.0
